In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join, expanduser
from tqdm import tqdm
import pandas as pd
import joblib
import sys
import numpy as np
from copy import deepcopy
from neuro.features.feat_select import get_alphas

In [ ]:
sparse_feats_dir = expanduser('~/mntv1/deep-fMRI/qa/sparse_feats_shared/')
os.listdir(sparse_feats_dir)

In [119]:
qa_sparse_feats_dir = join(
    sparse_feats_dir,
    # 'qa_embedder___qa_questions_version=v3_boostexamples___ensemble1')
    'eng1000___qa_questions_version=v1___mistralai-Mistral-7B-Instruct-v0.2')

In [123]:
seeds = range(5)
alphas = np.logspace(0, -3, 20)
nonzeros = pd.DataFrame(index=seeds, columns=alphas).astype(float)
enets = pd.DataFrame(index=seeds, columns=alphas)
fnames = os.listdir(qa_sparse_feats_dir)
for seed in tqdm(seeds):
    for alpha in alphas:
        template = f'seed={seed}___feature_selection_frac=0.50___feature_selection_alpha={alpha:.2e}.joblib'
        if template in fnames:
            coef_enet = joblib.load(join(qa_sparse_feats_dir, template))
            coef_enet_selected = deepcopy(
                np.any(np.abs(coef_enet) > 0, axis=0).squeeze())
            enets.loc[seed, alpha] = coef_enet_selected
            nonzeros.loc[seed, alpha] = coef_enet_selected.sum()
# template = f'seed={seed}___feature_selection_frac=0.50___feature_selection_alpha={feature_selection_alpha:.2e}.joblib'
# os.listdir(qa_sparse_feats_dir)

100%|██████████| 5/5 [01:02<00:00, 12.41s/it]


In [124]:
# v3_boost_examples has 674, eng1000 has 985
nonzeros.columns.name = 'alpha'
nonzeros.index.name = 'seed'
# nonzeros.columns = np.arange(len(nonzeros.columns))
# enets.columns = nonzeros.columns
# nonzeros.columns = nonzeros.columns.round(4)

# add row for fracs
for col in nonzeros.columns:
    coefs_list = enets[col]
    coefs_list = coefs_list[coefs_list.notna()]

    if len(coefs_list) > 0:
        # get fraction of times each element is True
        coefs_arr = np.vstack(coefs_list.values).astype(int)
        coefs_arr = coefs_arr.min(axis=0)
        nonzeros_stable = np.sum(coefs_arr)
    else:
        nonzeros_stable = np.nan
    nonzeros.loc['stable', col] = nonzeros_stable

display(
    nonzeros
    .style
    .background_gradient(cmap='viridis', axis=None)
    .format('{:.0f}')
)

alpha,1.000000,0.695193,0.483293,0.335982,0.233572,0.162378,0.112884,0.078476,0.054556,0.037927,0.026367,0.018330,0.012743,0.008859,0.006158,0.004281,0.002976,0.002069,0.001438,0.001000
seed,,,,,,,,,,,,,,,,,,,,
0,nan,nan,0,nan,nan,nan,227,344,nan,nan,896,nan,nan,nan,nan,nan,985,nan,nan,nan
1,0,nan,0,13,53,148,nan,nan,562,nan,892,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,nan,nan,nan,nan,58,144,233,351,nan,nan,nan,nan,979,985,985,nan,nan,985,nan,nan
3,0,0,0,nan,56,nan,230,346,554,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,nan,0,nan,nan,nan,153,224,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,985,nan,nan
stable,0,0,0,13,26,96,178,305,527,nan,887,nan,979,985,985,nan,985,985,nan,nan
